# Single Transferable Vote
## Counting Tool using Droop and GSheets
Based on https://github.com/jklundell/droop

And https://github.com/custozza/droop_from_gspreadsheet

Checkout submodules: `git submodule update --init --recursive`

# Setup

In [ ]:
# !pip install requests 
!pip install requests pandas numpy

import requests
import pandas as pd
import numpy as np
import re
import subprocess

In [ ]:
# uncomment to install droop, running with python works anyway
# !pip install -e deps/droop/setup.py

In [181]:
# Update these:
SPREADSHEET_ID = '1ZaN_eCVB2kFFkmeShoGR--mXGxCig28xbw_maABR7zQ' # ID from your public spreadsheet URL
NUM_SEATS = 14
IGNORE_CANDIDATES = []

VOTE_FILE = f'vote_seats_{NUM_SEATS}.csv'
BLT_FILE = f'vote_seats_{NUM_SEATS}.blt'
PATH_TO_DROOP = "deps/droop"
PYTHON_COMMAND = "python"
ELECTION_TITLE = "S8_Konferenz_2023"
LOG_FILE = f'election_seats_{NUM_SEATS}.log'
ELECTED_FILE = f'elected_seats_{NUM_SEATS}.txt'

# Reading and Parsing

FETCH RESULT FROM SPREAD SHEET

In [182]:
url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?exportFormat=csv"

response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the content of the CSV
    csv_data = response.text

    # Process or print the CSV data as needed
    with open(VOTE_FILE, 'w', encoding="utf-8") as f:
        print(csv_data, file=f)
    print(f'Downloaded vote data from {url} to {VOTE_FILE})')
else:
    print(f'Failed to fetch data. Status code: {response.status_code}')

CONVERT BALLOTS MATRIX TO SORTED CANDIDATES ID PER BALLOT

In [183]:
# CONVERT BALLOTS MATRIX TO SORTED CANDIDATES ID PER BALLOT

df = pd.read_csv(VOTE_FILE)

# remove votes for ignored candidates
if IGNORE_CANDIDATES:
    print('before dropping ignored candidates:')
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 500)
    print(df)
    unknown_ignored_candidates = [c for c in IGNORE_CANDIDATES if c not in df.values[:, 0]]
    if unknown_ignored_candidates:
        raise ValueError(f"Unknown candidates: {unknown_ignored_candidates}")
    ignore_ids = [i for i, c in enumerate(df.values[:, 0]) if c in IGNORE_CANDIDATES]
    # shift the indexing, large->small to avoid changing the index of the next candidate
    ignore_ids_desc = sorted(ignore_ids, reverse=True)
    for candidate_to_ignore in ignore_ids_desc:
        position = df.iloc[candidate_to_ignore, 1:].to_numpy().astype(np.float32)
        position_bc = np.broadcast_to(position, (len(df), len(position)))
        needs_shift = df.iloc[:, 1:] > position_bc
        position_shift = np.where(needs_shift, -1, 0)
        df.iloc[:, 1:] += position_shift
    # set rows with ignored candidates to NaN
    df.iloc[ignore_ids, 1:] = np.nan
    # remove ignored candidates
    df = df.drop(df.index[ignore_ids])

    print('\nafter dropping ignored candidates:')
    print(df)

candidates = df.values[:, 0]
ballots = df.values[:, 1:]

ballots_with_candidate_preferences = [
                                        [(candidate, pref) 
                                        for candidate, pref in enumerate(ballot) 
                                        if not np.isnan(pref)] 
                                        for ballot in ballots.T]

ballots_with_candidates_sorted_by_preference = [
                                        [candidate 
                                        for candidate, preference in sorted(ballot, key=lambda x: x[1])] 
                                        for ballot in ballots_with_candidate_preferences]

print(f'Number of candidates: {len(candidates)}, Number of ballots: {len(ballots_with_candidates_sorted_by_preference)}')
print('\n'.join([' '.join(str(b)) for b in ballots_with_candidates_sorted_by_preference]))


WRITE BLT

In [ ]:
blt_ballots = "\n".join(
    [f"1 {' '.join([str(c+1) for c in ballot])} 0 #" + ", ".join(candidates[c] for c in ballot)
     for ballot in ballots_with_candidates_sorted_by_preference])

blt_candidates = "\n".join(f'"{candidate} #{i}"' for i, candidate in enumerate(candidates,1))

with open(BLT_FILE, 'w') as file:
    blt_content = f'''
{len(candidates)} {NUM_SEATS}
{blt_ballots}
0 # end marker
{blt_candidates}
"{ELECTION_TITLE}" #Titel
    '''
    file.write(blt_content)
    print(blt_content)

EXECUTE DROOP

In [ ]:
# Test call directly
# !python deps/droop/Droop.py meek vote.blt

command = [PYTHON_COMMAND, PATH_TO_DROOP+'/Droop.py', "meek", BLT_FILE]
print(' '.join(command))
result = subprocess.run(command, capture_output=True, text=True)
with open(LOG_FILE, 'w') as file:
    file.write(result.stdout)

number_of_votes = len(ballots[0])
print('Ballots:', number_of_votes, 'Candidates:', len(candidates), 'Winners:', NUM_SEATS)
rounds = result.stdout.split('Round ')
election_pattern= r'Action: Elect: (?P<candidate>.*)'
electees_in_rounds = [(i, elected.group('candidate')) 
                      for i, r in enumerate(rounds[1:],1)
                      for elected in re.finditer(election_pattern, r) 
                       if elected ]

electees_in_rounds = [(id, round, person) for id, (round, person) in enumerate(electees_in_rounds, 1)]

electees_in_round = "\n".join(f"{str(id).rjust(2)}{str(round).rjust(5)} {person}" for id, round, person in electees_in_rounds)
elected_text = f"""
ID ROUND NAME
{electees_in_round}
"""
with open(ELECTED_FILE, 'w') as file:
    file.write(elected_text)
    print(elected_text)